In [4]:
import pandas as pd

races = pd.read_csv('race.csv')
runners = pd.read_csv('ultra_rankings.csv')

In [7]:
print(len(runners))
print(len(races))

137803
1207


In [8]:
runners.head()

,race_year_id,rank,runner,time,age,gender,nationality,time_in_seconds
0,68140,1.0,VERHEUL Jasper,26H 35M 25S,30,M,GBR,95725.0
1,68140,2.0,MOULDING JON,27H 0M 29S,43,M,GBR,97229.0
2,68140,3.0,RICHARDSON Phill,28H 49M 7S,38,M,GBR,103747.0
3,68140,4.0,DYSON Fiona,30H 53M 37S,55,W,GBR,111217.0
4,68140,5.0,FRONTERAS Karen,32H 46M 21S,48,W,GBR,117981.0


In [17]:
races.head()
print(races['distance'].max())
print(races['distance'].mean())
print(races['distance'].min())


179.1
152.61869096934527
0.0


In [20]:
total = pd.merge(runners,races, on='race_year_id')
total['average_speed'] = total['distance']/(total['time_in_seconds']/60)
total.head()

0         0.104612
1         0.102994
2         0.096523
3         0.090040
4         0.084878
            ...   
137798    0.075622
137799    0.075339
137800    0.075023
137801    0.074953
137802    0.074849
Name: average_speed, Length: 137803, dtype: float64

In [24]:
import numpy as np
np.histogram(total['average_speed'],bins=[0, 1, 2, 3])

(array([120011,      0,      1]), array([0, 1, 2, 3]))